# Stochastic Simulation

*Winter Semester 2023/24*

24.11.2023

Prof. Sebastian Krumscheid<br>
Asstistant: Stjepan Salatovic

<h3 align="center">
Exercise sheet 04
</h3>

---

<h1 align="center">
Stochastic process generation
</h1>

In [1]:
import matplotlib.pylab as plt
import numpy as np

from ipywidgets import interact
from scipy.fft import fft, ifft
from scipy.stats import uniform, norm, expon

In [2]:
plt.rc('axes', labelsize=14)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=12)    # fontsize of the tick labels
plt.rc('ytick', labelsize=12)    # fontsize of the tick labels
plt.rc('legend', fontsize=14)    # legend fontsize

## Exercise 1

Consider a fractional Brownian motion (fBM) $\{B^H(t), t\in[0,1]\}$,  which is a centred Gaussian process with $B^H(0)=0$ and covariance function 

$$
\mathrm{Cov}(t,s) = \frac{1}{2}(|t|^{2H} + |s|^{2H} - |t-s|^{2H}),
$$

where $H\in (0,1)$ is the so-called _Hurst index_. 

1. To sample such a process let us consider, for a fixed $h>0$, the increment process $\delta B_h(t) = B^H(t+h)-B^H(t)$. Show that $\delta B_h(t)$ is a centered stationary Gaussian process.

We begin by obtaining the covariance matrix for the incremental process $\delta B^H$. For any $t,s,h \geq 0$ we have

\begin{align*}
    \mathrm{Cov}_{\delta B^H}(t,s) &:= \mathrm{Cov}[B^H(t+h)-B^H(t),B^H(s+h)-B^H(s)]\\
    &=\mathbb{E}\left[\left(B^H(t+h)-B^H(t)\right)\left(B^H(s+h)-B^H(s)\right)\right]\\
    &=\mathbb{E}\left[ B^H(t+h)B^H(s+h)\right]-\mathbb{E}\left[B^H(t+h)B^H(s)\right]-\mathbb{E}\left[B^H(t)B^H(s+h)\right]+\mathbb{E}\left[B^H(s)B^H(t)		\right]\\
    &=\mathrm{Cov}_{\delta B^H}(t+h,s+h)-\mathrm{Cov}_{\delta B^H}(t+h,s)-\mathrm{Cov}_{\delta B^H}(t,s+h)+\mathrm{Cov}_{\delta B^H}(t,s)\\
    &=\frac{1}{2}\left(	|t+h|^{2H}	+|s+h|^{2H}-|t-s|^{2H}-|t+h|^{2H}-|s|^{2H}+|t+h-s|^{2H}-|t|^{2H}\right.\\ &-\left.|s+h|^{2H}+|s+h-t|^{2H}+|t|^{2H}+|s|^{2H}-|t-s|^{2H}	\right)\\
    &=\frac{1}{2}\left(|t+h-s|^{2H}+|s+h-t|^{2H}-2|t-s|^{2H}\right),
    \end{align*}

which is a stationary Gaussian process. Notice, moreover, that since  $\mathbb{E}[B^H(t)]=0, \ \forall t\geq 0$, then $\mathbb{E}[\delta B^H]=\mathbb{E}[B^H(t+h)]-\mathbb{E}[B^H(t)]=0$, and as such, $\delta B^H$ is a centered stationary Gaussian process. 

2. If one is able to sample exactly the process $\delta B_h(t)$ on a
	uniform grid $t_j=jh$, then one can construct an exact sample of the
	fractional Brownian motion on the same grid points as $B^H(t_k) =
	\sum_{j=0}^{k-1}\delta B_h(t_j)$. Sample a fractional Brownian motion using FFT and circular embedding. Implement your experiment for different values of $H<1/2$ and $H>1/2$.

    **Hint:** Have a look at Scipy's FFT module [`scipy.fft`](https://docs.scipy.org/doc/scipy/tutorial/fft.html).

In [3]:
def cov(t, s, h: float, H: float):
    """
    Covariance function of the fractional Gaussian noise with Hurst index `H`.
    """
    H2 = 2 * H
    return (np.abs(t + h - s) ** H2 + np.abs(s + h - t) ** H2 - 2 * np.abs(t - s) ** H2) / 2    

In [4]:
def fractional_Brownian_motion(n: int, H: float) -> np.array:
    """
    Samples one realization of a fractional Brownian motion on a
    uniform grid in [0, 1] with `n` points and Hurst index `H`.
    """
    h = 1 / n
    covs = cov(np.zeros(n + 1), np.arange(n + 1) * h, h, H)
    alpha = np.hstack((covs, covs[1:-1][::-1]))
    lambda_ = fft(alpha)
    y = np.random.randn(2 * n) + np.random.randn(2 * n) * 1j
    x_tilde = ifft(np.sqrt(2 * n) * np.sqrt(lambda_) * y)
    x1 = np.real(x_tilde[1:n])
    fBm = np.hstack((np.zeros(1), np.cumsum(x1)))
    return fBm

In [5]:
def plot_fBm(n, H):
    """Interaction helper."""
    np.random.seed(999)

    t = np.linspace(0, 1, n)
    fBm = fractional_Brownian_motion(n, H)

    plt.plot(t, fBm)
    plt.xlabel(r"$t$")
    plt.ylabel(r"$B^H(t)$")
    plt.title(rf"Fractional Brownian motion with Hurst index $H = {H:.1f}$");

In [6]:
interact(plot_fBm, n=(10, 1000), H=(0.1, 0.9));

interactive(children=(IntSlider(value=505, description='n', max=1000, min=10), FloatSlider(value=0.5, descript…

## Exercise 2

1. Generate a random walk $\{X_n\in\mathbb{Z},\,n\in\mathbb{N}_0,\,X_0=0\}$
  with transition probabilities
  \begin{equation*}
    \mathbb{P}(X_{n+1} = j \vert X_n = j-1) = \mathbb{P}(X_{n+1} = j \vert X_n = j+1) = a\;,\quad
    \mathbb{P}(X_{n+1} = j \vert X_n = j) = 1-2a\;,
  \end{equation*}
  for some $0<a\le 1/2$. 

In [7]:
def random_walk(n: int, a: float=.5) -> np.array:
    """
    Generates a random walk with `n` steps and transition probabilities specified by `a`.
    """
    u = np.random.rand(n)
    incr = 1 * (u <= a) - (u > a) * (u <= 2 * a)
    x = np.hstack((np.zeros(1), incr)).cumsum()
    return x

In [8]:
def plot_random_walk(n: int, a: float=.5):
    """Interaction helper."""
    np.random.seed(999)

    x = random_walk(n, a=a)

    plt.figure(figsize=(12, 4))
    plt.plot(x)
    plt.xlabel(r"$n$", size=14)
    plt.ylabel(r"$X_n$", size=14)
    plt.title(rf"Random walk for $a = {a}$")

In [9]:
interact(plot_random_walk, n=(10, 10000), a=(0, 0.5));

interactive(children=(IntSlider(value=5005, description='n', max=10000, min=10), FloatSlider(value=0.5, descri…

2. Consider the rescaled process $Y_{t_i} := \sqrt{\Delta t/(2a)}X_i$
  for $i=0,\dots, n$ with $t_i = i\Delta t$. Compare this process with
  the process $W_{t_i}$, $i=0,\dots, n$, where $W_t$ denotes a Wiener
  process with $W_0 = 0$. That is, show that both processes "look
  similar" in the limit as $\Delta t\to 0$ by plotting multiple
  realizations of both processes for $n = \lceil 1/\Delta t \rceil$.

In [10]:
def rescale_random_walk(x: np.array, a: float=.5) -> np.array:
    """
    Rescales the random walk `x` onto [0, 1] using transition probabilities specified by `a`.
    """
    dt = 1 / len(x)
    y = np.sqrt(dt / 2 / a) * x
    return y

In [11]:
def wiener_process(n: int) -> np.array:
    """Generates a Wiener process wih `n` points."""
    dt = 1 / n
    w = np.zeros(n + 1)
    w[1:] = np.sqrt(dt) * np.random.randn(n).cumsum()
    return w

In [12]:
def compare_rescaled_rw_and_wp(n: int, a: float=.5):
    """Interaction helper."""
    t = np.linspace(0, 1, n + 1)

    x = random_walk(n, a)
    y = rescale_random_walk(x, a=a)
    w = wiener_process(n)
    
    plt.figure(figsize=(12, 4))
    plt.plot(t, y, label=r"Rescaled Random walk $Y_t$")
    plt.plot(t, w, label=r"Wiener process $W_t$")
    plt.legend()
    plt.xlabel(r"$t$")
    plt.title(rf"Rescaled random walk compared to a Wiener process")

In [13]:
interact(compare_rescaled_rw_and_wp, n=(10, 10000), a=(0, 0.5));

interactive(children=(IntSlider(value=5005, description='n', max=10000, min=10), FloatSlider(value=0.5, descri…

3. **(Optional):** More theoretical analysis of the observed phenomenon:

    1. Consider the spatial mesh $x_m = m\Delta x = m \sqrt{\Delta t/(2a)}$ for
    $m \in \mathbb{Z}$ and the following notation for the rescaled
    process' probability mass function at time $t_i$:
    
    $$
      \bar u(t_i,x_m) := \mathbb{P}(Y_{t_i} = x_m|Y_{0} = 0), \quad m \in \mathbb{Z}, i =0,1,\ldots
    $$
    
    Use the discrete Chapman-Kolmogorov formula
    
    $$
       \tag{1}
      \mathbb{P}(Y_{t_{i+1}} = x_m|Y_{0} = 0) = \sum_{k} \mathbb{P}(Y_{t_{i+1}} = x_m|Y_{t_i} = x_k)\mathbb{P}(Y_{t_{i}} = x_k|Y_{0} = 0)
    $$
    
    to derive a difference equation for $\bar u(t_{i+1},x_m)$ in terms of
    $\bar u(t_{i},\cdot)$.

   2. Show that the difference equation obtained in $(1)$ corresponds to a finite difference approximation of the one dimensional heat equation
   
    $$
    u_t(t,x) = \frac{u_{xx}(t,x)}{2}, \quad x \in \mathbb{R}, t > 0,
    $$
       on a uniform grid $x_i = i \Delta x$ and $t_j = j \Delta t$ with $\Delta t = 2 a x^2$, using a second order centered finite difference stencil in space and a first order forward Euler scheme in time.

   3. For the standard Wiener process with $\mathbb{P}(W_0 =0)=1$,
    we denote the probability density function at time $t>0$ by
    $$
      u(t,x) := \frac{e^{-x^2/(2t)}}{\sqrt{2\pi t}}, \qquad x \in \mathbb{R}.
    $$
    For all $t> 0$ and $x \in \mathbb{R}$, show that the density satisfies the same heat equation introduced in point B.

A. We have that,

$$
\begin{align}
\bar{u}(t_{i+1}, x_m) &= \mathbb{P}(Y_{t_{i+1}} = x_m | Y_0 = 0)\\
&= \sum_{k \in \mathbb{Z}} \mathbb{P}(Y_{t_{i+1}} = x_m | Y_{t_i} = x_k) \mathbb{P}(Y_{t_{i}} = x_k | Y_0 = 0)\\
&= \sum_{k \in \mathbb{Z}} \mathbb{P}(Y_{t_{i+1}} = x_m | Y_{t_i} = x_k) \bar{u}(t_i,x_k)\\
&= (1-2a) \bar{u}(t_i,x_m) + a \bar{u}(t_i,x_{m-1}) + a \bar{u}(t_i,x_{m+1}) \tag{2}
\end{align}
$$

B. Using the Taylor series for the solution of the heat equation, we have that
$$
\begin{align*}
u(t_{i}, x_{m-1}) &= u(t_{i}, x_m) - \Delta x u_x(t_{i}, x_m) + \frac{\Delta x^2}{2} u_{xx}(t_{i}, x_m) + \mathcal{O}(\Delta x^4),\\
u(t_{i}, x_{m+1}) &= u(t_{i}, x_m) + \Delta x u_x(t_{i}, x_m) + \frac{\Delta x^2}{2} u_{xx}(t_{i}, x_m) + \mathcal{O}(\Delta x^4),\\
u(t_{i+1},x_m) &= u(t_{i},x_m) + \Delta t u_t(t_i,x_m) + \mathcal{O}(\Delta t^2).
\end{align*}
$$
With the choice $\Delta x=\sqrt{\Delta t/2a}$, this leads to
$$
\begin{align*}
u(t_{i+1},x_m ) &= u(t_i,x_m) + \frac{\Delta t}{2} u_{xx}(t_i,x_m) + \mathcal{O}(\Delta t^2)\\
&= u(t_i,x_m) + \frac{\Delta t}{2} \frac{u(t_i,x_{m-1})+u(t_i,x_{m+1})-2u(t_i,x_{m})}{\Delta x^2}+ \mathcal{O}(\Delta t^2 + \Delta x^4)\\
&= (1-2a)u(t_i,x_m) +a u(t_i,x_{m-1}) + u(t_i,x_{m+1})+ \mathcal{O}(\Delta t^2),
\end{align*}
$$
which corresponds to Eq. (2), up to error terms that vanish as $\Delta t\to 0$. 

C. This can be seen by simply substituting the density into the standard heat equation.

## Exercise 3

**Note:** Refer to Section **4.3** of the lecture notes.

Consider the chemical reactions between three species $S_1$, $S_2$,
$S_3$, which are determined by the following four reaction channels:
\begin{equation*}
  \begin{aligned}
    S_1 &\overset{c_1}{\to} \emptyset\;,\\
    S_1 + S_1  &\overset{c_2}{\to} S_2\;,\\
    S_2 &\overset{c_3}{\to} S_1 + S_1\;,\\
    S_2 &\overset{c_4}{\to} S_3\;.
  \end{aligned}
\end{equation*}
To simulate this system, consider the process
$\boldsymbol{N}_t = (N_t^1,N_t^2,N_t^3)\in\mathbb{N}_0^3$, where
$N_t^i$ denotes the number of molecules of species $S_i$ at time
$t\ge 0$. In fact, this process is a time-continuous Markov chain with
transition probabilities given by
\begin{equation*}
  \begin{aligned}
    \mathbb{P}\bigl(\boldsymbol{N}_{t+h} = \boldsymbol{N}_{t,1} = (N^1-1,N^2,N^3)\bigl\vert\bigr. \boldsymbol{N}_{t} &= (N^1,N^2,N^3)\bigr) = a_1(\boldsymbol{N}_{t})h + o(h)\;,\\
    \mathbb{P}\bigl(\boldsymbol{N}_{t+h}= \boldsymbol{N}_{t,2} = (N^1-2,N^2+1,N^3)\bigl\vert\bigr. \boldsymbol{N}_{t} &= (N^1,N^2,N^3)\bigr) = a_2(\boldsymbol{N}_{t})h + o(h)\;,\\
    \mathbb{P}\bigl(\boldsymbol{N}_{t+h} = \boldsymbol{N}_{t,3}= (N^1+2,N^2-1,N^3)\bigl\vert\bigr. \boldsymbol{N}_{t} &= (N^1,N^2,N^3)\bigr) = a_3(\boldsymbol{N}_{t})h + o(h)\;,\\
    \mathbb{P}\bigl(\boldsymbol{N}_{t+h} = \boldsymbol{N}_{t,4} =(N^1,N^2-1,N^3+1)\bigl\vert\bigr. \boldsymbol{N}_{t} &= (N^1,N^2,N^3)\bigr) =  a_4(\boldsymbol{N}_{t})h + o(h)\;,\\
    \mathbb{P}\bigl(\boldsymbol{N}_{t+h} = \boldsymbol{N}_{t,5}= (N^1,N^2,N^3)\bigl\vert\bigr. \boldsymbol{N}_{t}  &= (N^1,N^2,N^3)\bigr) = 1 - h\sum_{j=1}^4a_j(\boldsymbol{N}_{t})+ o(h)\;,\\
  \end{aligned}
\end{equation*}
for $h$ sufficiently small, where $\boldsymbol{N}_{t,k}, k \in \{1,...,5\}$ indexes the possible transitions. 
Here, the so-called propensity functions are
\begin{equation*}
  a_1(\boldsymbol{N}) = c_1 N^1\;,\quad a_2(\boldsymbol{N}) = c_2 \frac{N^1(N^1-1)}{2}\;,\quad a_3(\boldsymbol{N}) = c_3 N^2\;,\quad \quad a_4(\boldsymbol{N}) = c_4N^2\;,
\end{equation*}
with $\boldsymbol{N} = (N^1,N^2,N^3)$.


1. Try to construct the transition matrix corresponding to the above transition
  probabilities and note the challenges. Is it possible to simulate the chemical reaction without the explicit $Q$ matrix?

    **Hint:** Think back to how you simulated the process in Exercise 2.1.

Let $\bigl\{\boldsymbol{N}_t\in \mathbb{N}_0^3\colon t\in[0,T]\bigr\}$
be the Markov jump process that describes the number of each species
present in the chemical reaction system. That is, unlike the processes
covered during the lecture, we have to deal with both a vector-valued
process and the fact that the state space may be unbounded. Regardless
of these differences, the $Q$-matrix could be constructed as usual,
namely by
$Q = \bigl(q(\boldsymbol{n},\boldsymbol{m}),\,
\boldsymbol{n},\boldsymbol{m}\in \mathbb{N}_0^3\bigr)$, where

$$
\large
\begin{equation*}
  q(\boldsymbol{n},\boldsymbol{m}) = \begin{cases} \lim_{h\to 0}\frac{\mathbb{P}(\boldsymbol{N}_{t+h} = \boldsymbol{n}\vert \boldsymbol{N}_{t} = \boldsymbol{m})}{h}\;,& \boldsymbol{n} \not=\boldsymbol{m}\;,\\
    -\lim_{h\to 0}\frac{1-\mathbb{P}(\boldsymbol{N}_{t+h} = \boldsymbol{n}\vert \boldsymbol{N}_{t} = \boldsymbol{n})}{h}\;,& \boldsymbol{n} =\boldsymbol{m}\;.
    \end{cases}
\end{equation*}
$$

If one wanted to implement this matrix, then one would, of course,
have to truncate the state space appropriately. However, implementing
this matrix explicitly is neither needed nor advisable! In fact, the
code below shows an exemplary implementation. There, first the jump
times are generated, before its is determined which reaction takes
place.

2. Utilise the following algorithm to simulate the chemical reaction
  system. Plot a time series for each species' number of molecules for
  $t\in[0, T]$, $T=0.2$, for the reaction rates
  \begin{equation*}
    c_1 = 1\;,\quad c_2 = 5\;,\quad c_3 = 15\;,\quad c_4 = \frac{3}{4}\;,
  \end{equation*}
  using $\boldsymbol{N}_0 = (400,800,0)$ as initial number of
  molecules. Repeat the simulation for the same reaction rates
  $c_1,\dots, c_4$ also for $T=5$.

    **Algorithm 1:** Reaction simulation
    
    - Set $\boldsymbol{N}_0 = (N^1_0,N^2_0,N^3_0)$, $J_0=0$
    - **for** $n=1, 2, \ldots$ **do**
        - Compute $\lambda = \sum_{j=1}^4 a_j(\boldsymbol{N}_{J_{n-1}})$
        - Generate $S_n\sim \text{Exp}\left(\lambda\right)$ and set $J_n=J_{n-1}+S_n$
        - Generate $I \in \{1,2,3,4\}$ with probability mass function
          $$\mathbb{P}(I=j) = \frac{a_j(\boldsymbol{N}_{J_{n-1}})}{\sum_{l=1}^4 a_l(\boldsymbol{N}_{J_{n-1}})},
          $$
          which is the probability that the $j^{th}$ reaction happens.
        - Set $\boldsymbol{N}_t = \boldsymbol{N}_{J_{n-1}} \forall t \in [J_{n-1},J_n)$ and $\boldsymbol{N}_{J_n} = \boldsymbol{N}_{t,I}$
    - **end for**


In [29]:
def reaction_simulation(N0: np.array, c: np.array, T: float) -> np.array:
    """
    Reaction simulation for intial values `N0`, reaction rates `c` and time horizon `T`.
    Returns jump times as well as the process itself.
    """
    def transition(I: int) -> np.array:
        """State transition `I` in {1, 2, 3, 4}."""
        x, y, z = N[-1]
        if I == 0:
            x = x - 1
        elif I == 1:
            x, y = x - 2, y + 1
        elif I == 2:
            x, y = x + 2, y - 1
        elif I == 3:
            y, z = y - 1, z + 1
        N.append(np.array([x, y, z]))

    propensity = lambda N: c * np.array([N[0], N[0] * (N[0] - 1) / 2, N[1], N[1]])
    
    N = [N0]
    J = [0]
    
    while J[-1] < T:
        a = propensity(N[-1])
        lam = a.sum()
        S = expon(scale=1 / lam).rvs()
        J.append(J[-1] + S)
    
        u = np.random.rand()
        I = np.argmax(u <= np.cumsum(a / lam))
        transition(I)

    return J, N

In [35]:
def plot_reaction_simulation(N0: np.array, c: np.array, T: float):
    """Interaction helper."""
    J, N = reaction_simulation(N0, c, T)

    plt.plot(J, N, label=[r"$N^1$", r"$N^2$", r"$N^3$"])
    plt.xlabel(r"$t$", size=14)
    plt.ylabel(r"$N_t$", size=14)
    plt.legend(fontsize=12)
    plt.title("Reaction simulation")

In [36]:
c = np.array([1, 5, 15, 3 / 4])
N0 = np.array([400, 800, 0])
T = 1.0

In [37]:
interact(lambda T: plot_reaction_simulation(N0, c, T), T=[0.2, 5.0]);

interactive(children=(Dropdown(description='T', options=(0.2, 5.0), value=0.2), Output()), _dom_classes=('widg…

## Exercise 4

Let $\{N_t\in\mathbb{N}_0\colon t\ge 0,\, N_0=0\}$ be a Poisson process
with rate $\lambda$.

1. Show that, conditional on the event $\{N_T = n\}$, the jump
  times $J_1,\dots, J_n$ have joint density function
  \begin{equation*}
   f_{J_1,\dots,J_n}(j_1,\dots,j_n) =  n! \, T^{-n} \, \mathbb{I}(0\le j_1\le \dots\le j_n\le T)\;.
 \end{equation*}
 In other words, show that conditional on $\{N_T = n\}$, the jump
 times $J_1,\dots, J_n$ have the same distribution as an ordered sample of size $n$ from the uniform distribution on $[0, T]$.

    **Hints:** Use the joint distribution of the holding times
       $S_1,\dots,S_{n+1}$ to first derive the joint distribution of the
       jump times, where $S_{i+1} = J_{i+1}-J_i$. Then compute the conditional distribution of the jump
       times given that $N_T = n$, using the fact that
       $\{N_T = n\} = \{J_n\le T < J_{n+1}\}$ a.s.} 

Since the Poisson process
  $\{N_t\in\mathbb{N}_0\colon t\ge 0,\, N_0=0\}$ is non-decreasing,
  the condition $N_T=n$ implies that $n$ jumps need to take
  place. Let's denote these jump times by $J_1,\dots ,J_n$. As a
  matter of fact, the jump times are related to the $(n+1)$ holding
  times $S_1,\dots , S_{n+1}$ by $S_{i+1} = J_{i+1} - J_i$ for
  $i=0,\dots,n$, with the convention that $J_0 = 0$. We know that the
  holding times are i.i.d. $\text{Exp}(\lambda)$ for some
  $\lambda>0$. Consequently, their joint PDF reads
  \begin{equation*}
    f_{S_1,\dots , S_{n+1}}(s_1,\dots, s_{n+1}) = \lambda^{n+1}\exp{\biggl(-\lambda\sum_{i=1}^{n+1}s_i\biggr)}\mathbb{I}(s_1,\dots s_{n+1}\ge 0)\;.
  \end{equation*}
  As
  $\sum_{i=1}^{n+1}S_i = \sum_{i=1}^{n+1}(J_{i} - J_{i-1}) = J_{n+1}$,
  the joint PDF of the jumping times  $J_1,\dots ,J_{n+1}$ is given by
  \begin{equation*}
    f_{J_1,\dots , J_{n+1}}(t_1,\dots, t_{n+1}) = \lambda^{n+1}e^{(-\lambda t_{n+1})}
    \mathbb{I}(0\le t_1\le \dots\le t_{n+1})\;.
  \end{equation*}
  For any (Borel set) $A\subseteq \mathbb{R}^n$ we thus find  
  \begin{equation*}
    \begin{aligned}
      \mathbb{P} & \bigl((J_1,\dots , J_{n})\in A\vert N_T = n) = \frac{\mathbb{P}\bigl((J_1,\dots , J_{n})\in A\,,\, N_T = n)}{\mathbb{P}(N_T=n)}\\
      &= \mathbb{P}((J_1,...,J_n) \in A, J_n \leq T < J_{n+1})\\      
      &= \frac{n! T^{-n}}{\lambda^n e^{- \lambda T}} \int_T^{\infty} \left( \int_{A} \lambda^{n+1} e^{-\lambda t_{n+1}} \mathbb{I}(0\leq t_1 \leq ... \leq  t_n \leq T) dt_1 dt_2...dt_n\right)dt_{n+1}\\
      &= \frac{\lambda n! T^{-n}}{e^{- \lambda T}} \int_T^{\infty} e^{-\lambda t_{n+1}} dt_{n+1} \int_{A} \mathbb{I}(0\leq t_1 \leq ... \leq  t_n \leq T) dt_1 dt_2...dt_n\\
      &= n! T^{-n}\int_{A}\mathbb{I}(0\le t_1\le \dots\le t_{n}\leq T)\,dt_1\dots dt_n\;,
    \end{aligned}
  \end{equation*}
  as claimed.


2. Use the property above to propose an algorithm to generate the
  process $N_t$, $t\in (t_1,t_2)$, conditional upon $N_{t_1} = n_1$
  and $N_{t_2} = n_2>n_1$. Such a process is called _Poisson bridge_.

In [18]:
def poisson_bridge(t1: float, t2: float, n1: int, n2: int) -> np.array:
    """Generates a Poisson bridge between (`t1`, `n1`) and (`t2`, `n2`)."""
    assert n1 < n2, "n1 should be smaller than n2."

    dt = t2 - t1
    dn = n2 - n1

    J = np.sort(dt * np.random.rand(dn))
    
    t = t1 + np.hstack([0., J])

    t = np.hstack([t, t2])
    N = n1 + np.arange(dn + 1)
    N = np.hstack([N, n2])
    return t, N

In [19]:
def plot_poisson_bridge(t1: float, t2: float, n1: int, n2: int):
    """Interaction helper."""
    t, N = poisson_bridge(t1, t2, n1, n2)

    plt.step(t, N)
    plt.plot([t1, t2], [n1, n2], "ro", label=r"$n_1$ and $n_2$")
    plt.xlabel("t", size=14)
    plt.ylabel(r"$N_t$", size=14)
    plt.title("Poisson bridge")
    plt.legend(fontsize=12)

In [39]:
interact(plot_poisson_bridge, t1=(0, 10), t2=(0, 20), n1=(1, 10), n2=(10, 100));

interactive(children=(IntSlider(value=5, description='t1', max=10), IntSlider(value=10, description='t2', max=…